# EDA

In [1]:
import psycopg2 as ps
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
def conexion_BBDD(nombre_BBDD, usuario, contraseña, anfitrion, puerto):

    conn = ps.connect(
                    dbname = nombre_BBDD, 
                    user = usuario,
                    password = contraseña,
                    host = anfitrion,
                    port = puerto)

    return conn

In [3]:
dbname = "BBDD_Hipica", # base a la que nos queremos conectar
user = "postgres",
password = "admin",
host = "localhost",
port = "5432" # puerto en el que s eencuentra postgres

In [4]:
conn = conexion_BBDD(dbname[0], user[0], password[0], host[0], port)
cur = conn.cursor()
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Análisis de concursos y pruebas

In [5]:
def ejecutor_querys(cur, query):
    cur.execute(query)
    return cur.fetchall()

In [ ]:
query_concursos_completos = """
        SELECT *
        FROM concursos c;
"""
query_n_concursos = """
        SELECT COUNT(id_concurso)
        FROM concursos c;
"""
query_fechas_concursos = """
        SELECT 
                MIN(fecha_inicio_concurso),
                MAX(fecha_fin_concurso)
        FROM concursos c;
"""
query_concursos_mes = """
        SELECT 
                EXTRACT(MONTH FROM fecha_inicio_concurso),
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY EXTRACT(MONTH FROM fecha_inicio_concurso)
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_ambito = """
        SELECT 
                ambito_concurso,
                CONCAT((COUNT(id_concurso)) * 100 / (SELECT COUNT(id_concurso) FROM concursos c), ' ', '%')
        FROM concursos c
        GROUP BY ambito_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_federacion = """
        SELECT 
                federacion_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY federacion_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_provincia = """
        SELECT 
                provincia_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY provincia_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""

query_concursos_localidad = """
        SELECT 
                localidad_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY localidad_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""

query_concursos_localidad = """
        SELECT 
                localidad_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY localidad_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_pruebas_concurso = """
        
        WITH pruebas_concurso AS (
                SELECT 
                        c.id_concurso,
                        COUNT(DISTINCT p.id_prueba) as n_pruebas_concurso
                FROM resultados r
                        JOIN concursos c ON r.id_concurso = c.id_concurso
                        JOIN pruebas p ON r.id_prueba = p.id_prueba
                GROUP BY c.id_concurso
        )
        SELECT 
                ROUND(AVG(n_pruebas_concurso), 0)
        FROM pruebas_concurso;

"""
# para tipos de pruebas: SELECT CHARINDEX('OM', 'Customer') AS MatchPosition;

In [57]:
meses = {
    1: "enero",
    2: "febrero",
    3: "marzo",
    4: "abril",
    5: "mayo",
    6: "junio",
    7: "julio",
    8: "agosto",
    9: "septiembre",
    10: "octubre",
    11: "noviembre",
    12: "diciembre"
}


In [143]:
conn.rollback()

In [131]:
federaciones = ejecutor_querys(cur, query_concursos_federacion)

In [132]:
concursos_provincias = ejecutor_querys(cur, query_concursos_provincia)

In [133]:
localidades = ejecutor_querys(cur, query_concursos_localidad)

In [161]:
n_concursos = ejecutor_querys(cur, query_n_concursos)
fechas_concursos_25 = ejecutor_querys(cur, query_fechas_concursos)
n_concursos_mes = ejecutor_querys(cur, query_concursos_mes)
n_concursos_ambito= ejecutor_querys(cur, query_concursos_ambito)
media_pruebas_concurso = ejecutor_querys(cur, query_concursos_pruebas_concurso)
print(f"""
      Se han celebrado un total de {n_concursos[0][0]} concursos, entre el {str(fechas_concursos_25[0][0])} y el {str(fechas_concursos_25[0][1])}. Un {n_concursos_ambito[0][1]} de los concursos han sido de 
      ámbito {str(n_concursos_ambito[0][0]).lower()}, frente a los de ámbito {str(n_concursos_ambito[1][0]).lower()}, que representan un {n_concursos_ambito[1][1]} del total.
      
      El mes que más concursos se han realizado ha sido {meses[int(n_concursos_mes[0][0])]}, con un total de {n_concursos_mes[0][1]} concursos, y el mes que menos concursos se han realizado ha sido el mes de {meses[int(n_concursos_mes[-1][0])]}, 
      con un total de {n_concursos_mes[-1][1]} concursos. La provincia en la que más concursos se han realizado ha sido {concursos_provincias[0][0]}, y la provincia en la que menos se han realizado ha sido {concursos_provincias[-1][0]}.
      
      De media en un concurso se realizan {str(media_pruebas_concurso[0][0]).replace('.', ',')} pruebas. 
      
      """)




      Se han celebrado un total de 122 concursos, entre el 2025-01-10 y el 2025-04-27. Un 73 % de los concursos han sido de 
      ámbito internacional, frente a los de ámbito nacional, que representan un 26 % del total.

      El mes que más concursos se han realizado ha sido febrero, con un total de 41 concursos, y el mes que menos concursos se han realizado ha sido el mes de abril, 
      con un total de 21 concursos. La provincia en la que más concursos se han realizado ha sido Valencia, y la provincia en la que menos se han realizado ha sido Córdoba.

      De media en un concurso se realizan 13 pruebas. 

      


In [126]:
concursos = pd.DataFrame(ejecutor_querys(cur, query_concursos_completos)).rename(columns = {0: "id_concurso", 1: "nombre_concurso", 2: "categoria_concurso", 3: "pais_concurso", 4: "provincia_concurso", 5: "localidad_concurso", 6: "ambito_concurso", 7: "federacion_concurso", 8: "fecha_inicio_concurso", 9: "fecha_fin_concurso"})
concursos.head()

,id_concurso,nombre_concurso,categoria_concurso,pais_concurso,provincia_concurso,localidad_concurso,ambito_concurso,federacion_concurso,fecha_inicio_concurso,fecha_fin_concurso
0,1,CSN3* Invierno,CSN***,Epaña,Sevilla,Real Club Pineda,Nacional,Federación Hípica Andaluza,2025-01-10,2025-01-12
1,2,SPRING MET I 2025 YH*,CSI YH*,Epaña,Valencia,Centro Ecuestre Oliva Nova,Internacional,Federación Hípica Comunidad Valenciana,2025-01-14,2025-01-16
2,3,MOURA TOURS VALENCIA - SPRING TOUR,CSI*,Epaña,Valencia,Moura Tours,Internacional,Federación Hípica Comunidad Valenciana,2025-01-20,2025-01-26
3,4,SPRING MET I 2025 - SILVER 1*,CSI*,Epaña,Valencia,Centro Ecuestre Oliva Nova,Internacional,Federación Hípica Comunidad Valenciana,2025-01-16,2025-01-18
4,5,SPRING MET I 2025 - GOLD 2*,CSI**,Epaña,Valencia,Centro Ecuestre Oliva Nova,Internacional,Federación Hípica Comunidad Valenciana,2025-01-16,2025-01-19


In [130]:
# Suponiendo que tu DataFrame se llama df
diccionario = concursos.groupby('provincia_concurso')['localidad_concurso'].unique().apply(list).to_dict()
diccionario

{'Asturias': ['Club Hipico Astur'],
 'Barcelona': ['Real Club De Polo De Barcelona', 'Open Sports Club'],
 'Cantabria': ['Heras Horses & Events'],
 'Cádiz': ['Dehesa Montenmedio'],
 'Córdoba': ['Palma Del Rio'],
 'Granada': ['Granada'],
 'Madrid': ['Club Hipico Las Cadenas',
  'R.A.C.E.',
  'Las Cadenas',
  'R.S.H.E.C.C.'],
 'Murcia': ['Club Aros'],
 'Segovia': ['Cenro Ecuestre De Castilla Y León'],
 'Sevilla': ['Real Club Pineda'],
 'Tenerife': ['C.H Nueva Atalaya', 'C.H. Del Sur Los Migueles'],
 'Valencia': ['Centro Ecuestre Oliva Nova',
  'Moura Tours',
  'Ceon',
  'Mouratours Valencia',
  'Mouratours'],
 'Zamora': ['Equus Duri']}

## Análisis de jinetes

## Análisis de caballos

## Análisis de pruebas

## Análisis de resultados